In [8]:
import simpy
import random

In [9]:
# First Step : Defining Ressources
# Defining an paper dispenser

class paperdispenser(object):
    def __init__(self, env, operator, robot, jig1, jig2, workstation1):
        self.env = env
        self.operator = operator
        self.robot = robot
        self.jig1 = jig1
        self.jig2 = jig2
        self.workstation1 = workstation1

    # Paper dispenser contains 100 papers, in which the papers are placed into the station 1.
    def check_availability_paperdispenser(self, operator):
        # check if there is enough paper on the dispenser
        # This takes between 7-10 seconds
        yield self.env.timeout(random.randint(7, 10))

        if self.paperdispenser == 1:
            # if there are enough paper, return True
            return True
        else:
            print("There are not enough paper available on the dispenser, please refill ")
            # TODO: trigger a new process while restocking is in progress
            return False

    def pick_paper(self, operator, workstation1):
        while not self.env.process(self.check_availability_paperdispenser(robot)):
            yield self.env.timeout(1)
            yield self.pickup_paperdispenser.get(1) & self.env.process(self.restock())  # & self.env.process(workstation3.pickup_paper(robot))

    # TODO: Trigger of the restock for the paperdispenser required?
    def restock_paperdispenser(self):
        # restock the paperdispenser , this takes between 20-30 seconds
        yield self.env.timeout(random.randint(20, 30))
        # restock the paper dispenser
        yield self.pickup_paperdispenser.put(100)

# ----------------------------------------------

# 2 Defining robot

class robot(object):
    def __init__(self, env, workstation1):
        self.env = env
        # the robot belongs to workstation
        self.workstation1 = workstation1
        self.robot = simpy.Resource(env, capacity=1)
        self.quality_check_jig1 = False
        self.quality_check_jig2 = False
        self.available = True
        self.at_start_position = False
        self.paper_loaded = False

    # Calibrating the robot , check the robot is in start position
    def actuate(self, workstation1):
        try:
            # check if the robot is available and the start position is required
            if self.available and self.at_start_position:
                # start the process of the robot
                self.env.process(self.start_process())
                # calibration takes place between 2-5seconds
                yield self.env.timeout(random.randint(2-5))
                self.available = False
            else:
                yield self.env.timeout(1)
        except Exception as e:
            # Error message if the robot is not available or the start position is not reached
            print(f"Error: {e}")
            # TODO: check if the error occurs permanently and adjust the error message if necessary
            yield self.env.process(self.eliminate_actuation_error(self))  # TODO: is this correct? Verweis auf class operator def eliminate_actuation_error bei Error

    def start_process(self):
        pass

    # paper from paperdispenser
    def load_paper(self, paper, paperdispenser):
        yield self.env.timeout(random.randint(3-5))

    # The paper is pushed into the Jig 1 by the robot
    def push_paper_to_jig1(self, jig1):
        yield self.env.timeout(random.randint(5, 10))
        yield self.env.process(jig1.folding_process_1(self))

    # Paper to jig 2 by robot
    def insert_paper_to_jig2(self, jig2):
        yield self.env.timeout(random.randint(5, 10))

    # extract the paper from the Jig 2 by robot
    def extract_paper_from_jig2(self, jig2):
        yield self.env.timeout(random.randint(1, 5))

    # place the paper on the station 2
    def hand_over_paper(self, workstation2):
        yield self.env.timeout(random.randint(1, 5))

    # The operation by the robot is finished , so the robot is returned into the initial position
    def return_to_start(self):
        self.check_paper_unloaded()
        yield self.env.timeout(10)
        self.at_start_position = True
        print("Robot is ready for next folding operation ")
        self.available = True

# ----------------------------------------------

# 3 Defining an operator class

class operator(object):
    def __init__(self, env, workstation1, qualitycheck1, qualitycheck2, paperdispenser):
        self.env = env
        self.qualitycheck1 = qualitycheck1
        self.qualitycheck2 = qualitycheck2
        self.wastestation1 = wastestation1
        self.wastestation2 = wastestation2
        self.workstation1 = workstation1
        self.operator = simply.Resource(env, capacity=1)
        self.available = True

    # Quality check 1
    def quality_check_fold1(self, paper):
        # Quality check after folding in Rig1
        yield self.env.timeout(random.randint(3, 7))

        if paper.is_defective:
            # If the paper is defective, send it to waste
            print("Paper from Rig1 has a defect and goes to waste.")
            yield self.env.process(self.workstation1.waste_station1.dispose(paper))
        else:
            print("Paper from Rig1 passed quality check.")

    def place_paper_on_robot1(self, robot, paper):
        yield self.env.timeout(random.randint(1, 5))
        print("Operator placed the folded paper on the robot.")

    # Quality check 2
    def quality_check_fold2(self, paper):
        # Quality check after folding in Rig2
        yield self.env.timeout(random.randint(3, 7))

        if paper.is_defective:
            # If the paper is defective, send it to waste
            print("Paper from Rig2 has a defect and1.waste_station2.dispose(paper)")
            yield self.env.process(self.workstation1.waste_station2.dispose(paper))
        else:
            print("Paper from Rig2 passed quality check.")

    def place_paper_on_robot2(self, robot, paper):
        yield self.env.timeout(random.randint(1, 5))
        print("Operator placed the folded paper  on the robot.")


#---------------------------------------------


#--------------------------------------------------------------------

#5 Jig 1

class Rig1(object):
    def __init__(self, env):
        self.env = env

    def fold1(self, paper1):
        # Folding process in Rig 1
        yield self.env.timeout(random.randint(5, 10))
        print("Paper has been folded.")

   
class Rig2(object):
    def __init__(self, env):
        self.env = env

    def fold2(self, paper2):
        # Folding process in Rig 2
        yield self.env.timeout(random.randint(5, 10))
        print("Paper has been folded.")

    
          #---------------------------

class WasteStation1(object):
    def __init__(self, env):
        self.env = env

    def dispose(self, paper1):
        # Waste disposal process
        yield self.env.timeout(random.randint(3, 7))
        print("Paper has been disposed of as waste.")
        
class WasteStation2(object):
    def __init__(self, env):
        self.env = env

    def dispose(self, paper2):
        # Waste disposal process
        yield self.env.timeout(random.randint(3, 7))
        print("Paper has been disposed of as waste.")
        
























    


    
    




    

In [10]:
# 2. Environment

class workstation1(object):
    def __init__(self, env, paper_list):
        self.env = env
        self.robot = robot(env, self)
        self.operator = operator(env, self, quality_check_fold1, quality_check_fold2, paperdispenser, wastestation1, wastestation2)
        self.jig1 = jig1(env, self)
        self.jig2 = jig2(env, self)
        self.paper = paper(env)
        self.produced_papers = 0
        self.total_number_of_folded_papers = 0
        self.total_production_time = 0
        self.paperdispenser = paperdispenser

    def workstation1_process(self, paper):
        start_time = self.env.now
        yield self.env.process(self.robot.acutate(self))
        print("Paper " + str(self.produced_papers) + " started the process at " + str(self.env.now) + ".")
        # At first the robot has to take paper from the paper dispenser
        yield self.env.process(self.robot.load_paper(self.paper, self, self))
        print("Robot " + str(self.produced_papers) + " loaded the paper at " + str(self.env.now) + ".")
        # Push the paper to the Jig 1
        yield self.env.process(self.robot.push_paper_to_jig1(self.jig1))
        print("Robot pushed the paper" + str(self.produced_papers) + " to jig1 at " + str(self.env.now) + ".")
        # Folding process in Jig1
        yield self.env.process(self.jig1.fold1(self.robot, self.paper))
        # Quality check after Jig1
        yield self.env.process(self.robot.quality_check_fold1(paper1))
        print("Operator checked the quality of fold 1 of the paper " + str(self.produced_papers) + " at " + str(self.env.now) + ".")
        # Place the folded paper onto the robot
        yield self.env.process(self.operator.place_paper_on_robot1(self.robot, paper1))
        # Insert the paper into Jig2
        yield self.env.process(self.robot.insert_paper_to_jig2(self.jig2))
        print("Robot inserted the paper " + str(self.produced_papers) + " into Jig2 at " + str(self.env.now) + ".")
        # Folding process in Jig2
        yield self.env.process(self.jig1.fold1(self.robot, self.paper))
        # Quality check after Jig2
        yield self.env.process(self.robot.quality_check_fold2(paper))
        print("Operator checked the quality of fold 2 of the paper " + str(self.produced_papers) + " at " + str(self.env.now) + ".")
        # Place the folded paper onto the robot
        yield self.env.process(self.operator.place_paper_on_robot2(self.robot, paper))
        # Handover the paper to the next station
        yield self.env.process(self.robot.hand_over_paper(self.station2))
        print("Robot completed handover to Station 2 at " + str(self.env.now) + ".")
        # Return the robot to the start position
        yield self.env.process(self.robot.return_to_start())
        print("Robot returned to the start position at " + str(self.env.now) + ".")
        yield self.env.process(self.robot.actuate(self))
        operation_time = self.env.now - start_time
        paper.operation_time = operation_time
        # print ("Process time of "+str(self.produced_papers)+" is "+str(self.env.now)+" seconds.")
        # add the process time to the total production time
        self.total_production_time += process_time
        # count the produced papers
        self.produced_papers += 1

    def paper_generator(env, paper_dispenser, workstation1, paper_list):
        # generate more papers while the simulation is running
        while True:
            # interarrival time of the papers is between 60 seconds and 100 seconds
            interarrival_time = random.randint(100, 120)

            # create a new paper
            paper_generator1 = paper(env, workstation1)
            paper_list.append(paper_generator1)

            yield env.timeout(interarrival_time)
            env.process(workstation1.workstation1_process(paper))

            # Wait for the next paper
            yield env.timeout(interarrival_time)

            print("Next Paper " + str(workstation1.produced_papers) + " arrived at Station 1 at " + str(env.now) + " seconds.")
# Environment

env = simpy.Environment()
Station_instance = Workstation1(env, [])
env.process(paper_generator(env, Station_instance, []))
env.run(until=28800)
# --------------------
# The simulation is finished, now we can analyze the results
print("The simulation is finished")
# Print the time the simulation runs
print("The simulation runs for " + str(env.now) + " seconds.")
# Print the number of produced papers
# Keep in mind that there is a paper in the process at the beginning of the simulation
print("The number of produced papers is " + str(Station_instance.produced_papers - 1) + ".")
# Average process time of the produced papers
print("The average process time of the produced papers is " + str(Station_instance.total_production_time / (Station_instance.produced_papers - 1)) + " seconds.")


NameError: name 'Workstation1' is not defined